In [2]:
!pip install alibi[shap]


  Obtaining dependency information for shap<0.44.0,>=0.40.0 from https://files.pythonhosted.org/packages/f5/fc/e81722d6bec4fcba46e46ef895eddaeab0027ac71e78fc35ef351fac5fe4/shap-0.43.0-cp311-cp311-win_amd64.whl.metadata
  Using cached shap-0.43.0-cp311-cp311-win_amd64.whl.metadata (24 kB)
  Obtaining dependency information for slicer==0.0.7 from https://files.pythonhosted.org/packages/78/c2/b3f55dfdb8af9812fdb9baf70cacf3b9e82e505b2bd4324d588888b81202/slicer-0.0.7-py3-none-any.whl.metadata
  Using cached slicer-0.0.7-py3-none-any.whl.metadata (3.7 kB)
Using cached shap-0.43.0-cp311-cp311-win_amd64.whl (447 kB)
Using cached slicer-0.0.7-py3-none-any.whl (14 kB)
  Attempting uninstall: slicer
    Found existing installation: slicer 0.0.8
    Uninstalling slicer-0.0.8:
      Successfully uninstalled slicer-0.0.8
  Attempting uninstall: shap
    Found existing installation: shap 0.46.0
    Uninstalling shap-0.46.0:
      Successfully uninstalled shap-0.46.0


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from alibi.explainers import TreeShap

# Kreirajte DataFrame sa primer podacima
data = {
    'size': ['S', 'M', 'L', 'XL', 'XXL', 'M', 'L', 'S', 'XXL', 'XL'],
    'age': [23, 45, 34, 30, 60, 52, 45, 31, 25, 40],
    'target': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
}

df = pd.DataFrame(data)

# Pretvorite kategoričke varijable u numeričke koristeći one-hot encoding
df_encoded = pd.get_dummies(df, columns=['size'])

# Podelite podatke na karakteristike i oznake
X = df_encoded.drop(columns='target')
y = df_encoded['target']

# Podelite podatke na obučeni i test skup
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardizujte podatke
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Obučite Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Inicijalizujte TreeShap objašnjavača
explainer = TreeShap(
    model,
    feature_names=X.columns.tolist()
)

# Fitujte objašnjavača sa obukom podataka
explainer.fit(X_train_scaled, summarise_background=True)

# Generišite objašnjenja za test set
explanations = explainer.explain(X_test_scaled)

# Proverite sadržaj 'shap_values'
shap_values = explanations.shap_values  # SHAP vrednosti se nalaze u ovom atributu

# Ispisujemo tip i dimenzije shap_values za bolje razumevanje
print(type(shap_values))
print(len(shap_values))
print(shap_values[0].shape)

# Prikaz SHAP vrednosti za jedan primer
sample_index = 0
shap_values_for_sample = shap_values[0][sample_index]  # Prvi element u shap_values je za klasu 0

# Prikaz SHAP vrednosti u DataFrame
shap_df = pd.DataFrame(shap_values_for_sample.reshape(1, -1), columns=X.columns)
print(f'SHAP vrednosti za primer {sample_index}:')
print(shap_df)


<class 'list'>
2
(3, 6)
SHAP vrednosti za primer 0:
        age    size_L  size_M    size_S   size_XL  size_XXL
0 -0.099762  0.061429     0.0  0.009762 -0.075952 -0.082619
